In [1]:
import json
from azure.eventhub import EventHubProducerClient, EventData
from dotenv import load_dotenv
import os
import socket
import random

from random import randrange

load_dotenv()
EVENT_HUB_CONN_STRING = os.getenv("EVENT_HUB_CONN_STRING") 
EVENT_HUB_NAME_IMPRESSIONS = os.getenv("EVENT_HUB_NAME_IMPRESSIONS")
EVENT_HUB_NAME_CLICKS = os.getenv("EVENT_HUB_NAME_CLICKS")

eventHubConnString = EVENT_HUB_CONN_STRING
eventHubNameImpressions = EVENT_HUB_NAME_IMPRESSIONS
eventHubNameClicks = EVENT_HUB_NAME_CLICKS

producer_impressions = EventHubProducerClient.from_connection_string(conn_str=eventHubConnString, eventhub_name=eventHubNameImpressions)
producer_clicks = EventHubProducerClient.from_connection_string(conn_str=eventHubConnString, eventhub_name=eventHubNameClicks)

hostname = socket.gethostname()

In [2]:
from utils import generateClickEvent, generateImpressionEvent

def sendToEventsHub(jsonEvent, producer):
    eventString = json.dumps(jsonEvent)
    print(eventString) 
    event_data_batch = producer.create_batch() 
    event_data_batch.add(EventData(eventString)) 
    producer.send_batch(event_data_batch)

def generateEvents(isUnsupportedBrowser = False, isAnomaly = False):
    try:
        while True:
            impressionEvent = generateImpressionEvent(isUnsupportedBrowser, isAnomaly, None)    
            sendToEventsHub(impressionEvent, producer_impressions)
            if random.randint(1, 100) > 80:
                clickEvent = generateClickEvent(impressionEvent)    
                sendToEventsHub(clickEvent, producer_clicks)
    except KeyboardInterrupt:
        producer_impressions.close()
        producer_clicks.close()


In [3]:
# generate anomalies in load time
generateEvents(False, True)

{"impressionId": "b8c0e7ec-450c-4f0b-a6c4-80cb7e3f750a", "impressionDate": "2024-01-03T00:21:36", "productId": 885, "browser": "Firefox", "browserVersion": "8.5", "device": "mobile", "source": "facebook", "ip_address": "104.197.171.50", "landing_page": "https://www.mccormick.com/tags/categorieshome.html", "page_loading_seconds": 43.73}
{"impressionId": "7489a773-f01b-406b-a7dd-60d2cd6e8e79", "impressionDate": "2024-01-07T07:05:14", "productId": 708, "browser": "Safari", "browserVersion": "11.2", "device": "mobile", "source": "bing", "ip_address": "17.201.82.73", "landing_page": "https://moore.com/wp-content/taghomepage.php", "page_loading_seconds": 28.66}
{"impressionId": "0ed16e59-e592-4276-9bb3-7c555f4aa485", "impressionDate": "2024-01-08T23:52:53", "productId": 964, "browser": "Edge", "browserVersion": "4.5", "device": "computer", "source": "facebook", "ip_address": "177.102.150.15", "landing_page": "https://www.rodriguez.org/search/postshomepage.html", "page_loading_seconds": 40.37